In [27]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
from itertools import combinations
from operator import itemgetter
import json
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
version = 'v2'

race_type = 'R'
r = 2

bet_type = 'E'

In [29]:
engine = create_engine('sqlite:///../../data/exotic.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('exotic', conn)
print('{} exotics loaded!'.format(len(dfoo)))

372604 exotics loaded!


In [30]:
dfo = dfoo.loc[dfoo['race_type'].isin([race_type])]
print('{} {} exotics!'.format(len(dfo), race_type))

372604 R exotics!


In [32]:
dfo.tail()

,id,race_type,bet_type,res1,res2,res3,res4,num_runners,run1_num,run1_win_perc,...,run4_num,run4_win_perc,run4_win_scaled,run4_win_rank,run4_place_perc,run4_place_scaled,run4_place_rank,prediction,success,dividend
372599,372600,R,Q,3,1,9,4,8.0,4,0.090909,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7.7
372600,372601,R,Q,3,1,9,4,8.0,4,0.090909,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7.7
372601,372602,R,Q,3,1,9,4,8.0,6,0.081967,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7.7
372602,372603,R,Q,3,1,9,4,8.0,6,0.081967,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7.7
372603,372604,R,Q,3,1,9,4,8.0,7,0.057471,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7.7


In [22]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    combs = build_combinations(race['runners'], r, race['results'], race['exacta'])
#     print(combs)
    if combs:
        data_all = data_all.append(combs)
#     break
data_all.tail()

,bet_type,dividend,num_runners,race_type,run1_num,run1_place_perc,run1_place_rank,run1_place_scaled,run1_win_perc,run1_win_rank,run1_win_scaled,run2_num,run2_place_perc,run2_place_rank,run2_place_scaled,run2_win_perc,run2_win_rank,run2_win_scaled,success
16,E,8.4,7.0,R,6,0.625000,1.000000,0.262295,0.136986,0.666667,0.116077,7,0.270270,0.444444,0.113425,0.064935,0.444444,0.055024,0
17,E,8.4,7.0,R,6,0.625000,1.000000,0.262295,0.136986,0.666667,0.116077,3,0.082645,0.333333,0.034684,0.042373,0.333333,0.035905,0
18,E,8.4,7.0,R,2,0.294118,0.666667,0.123433,0.135135,0.555556,0.114509,7,0.270270,0.444444,0.113425,0.064935,0.444444,0.055024,0
19,E,8.4,7.0,R,2,0.294118,0.666667,0.123433,0.135135,0.555556,0.114509,3,0.082645,0.333333,0.034684,0.042373,0.333333,0.035905,0
20,E,8.4,7.0,R,7,0.270270,0.444444,0.113425,0.064935,0.444444,0.055024,3,0.082645,0.333333,0.034684,0.042373,0.333333,0.035905,0


In [33]:
# get label data
Y = data_all['success']

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    323056.000000
mean          0.011475
std           0.106504
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: success, dtype: float64

In [34]:
X = data_all[[
    'num_runners',
    'run1_win_perc', 'run1_win_scaled', 'run1_win_rank', 'run1_place_perc', 'run1_place_scaled', 'run1_place_rank',
    'run2_win_perc', 'run2_win_scaled', 'run2_win_rank', 'run2_place_perc', 'run2_place_scaled', 'run2_place_rank'
]]
X.tail()

,num_runners,run1_win_perc,run1_win_scaled,run1_win_rank,run1_place_perc,run1_place_scaled,run1_place_rank,run2_win_perc,run2_win_scaled,run2_win_rank,run2_place_perc,run2_place_scaled,run2_place_rank
16,7.0,0.136986,0.116077,0.666667,0.625000,0.262295,1.000000,0.064935,0.055024,0.444444,0.270270,0.113425,0.444444
17,7.0,0.136986,0.116077,0.666667,0.625000,0.262295,1.000000,0.042373,0.035905,0.333333,0.082645,0.034684,0.333333
18,7.0,0.135135,0.114509,0.555556,0.294118,0.123433,0.666667,0.064935,0.055024,0.444444,0.270270,0.113425,0.444444
19,7.0,0.135135,0.114509,0.555556,0.294118,0.123433,0.666667,0.042373,0.035905,0.333333,0.082645,0.034684,0.333333
20,7.0,0.064935,0.055024,0.444444,0.270270,0.113425,0.444444,0.042373,0.035905,0.333333,0.082645,0.034684,0.333333


In [35]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 100
print('epochs = {}'.format(epochs))

layer_1 = 40
layer_2 = 40

tag = '{}{}x{}{}'.format(race_type, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/exotic/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 13
epochs = 100
tag = R40x40E
model loaded


In [36]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/exotic/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
